In [0]:
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
import transformer
import glob
import pickle
from torch.utils.data import TensorDataset, DataLoader

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# load the dataset
poems = []

for path in glob.iglob("drive/My Drive/datasets/poemas_machado/*.txt"):
    with open(path, 'r') as f:
        x = f.read()
        poems.append(x.lower().split(' '))
        
print(poems[0])

['a', 'don', 'miguel', 'de', 'unamuno', '\n', 'por', 'su', 'libro', 'vida', 'de', 'don', 'quijote', 'y', 'sancho', '.', '\n', 'este', 'donquijotesco', '\n', 'don', 'miguel', 'de', 'unamuno', ',', 'fuerte', 'vasco', ',', '\n', 'lleva', 'el', 'arnés', 'grotesco', '\n', 'y', 'el', 'irrisorio', 'casco', '\n', 'del', 'buen', 'manchego', '.', 'don', 'miguel', 'camina', ',', '\n', 'jinete', 'de', 'quimérica', 'montura', ',', '\n', 'metiendo', 'espuela', 'de', 'oro', 'a', 'su', 'locura', ',', '\n', 'sin', 'miedo', 'de', 'la', 'lengua', 'que', 'malsina', '.', '\n', 'a', 'un', 'pueblo', 'de', 'arrieros', ',', '\n', 'lechuzos', 'y', 'tahúres', 'y', 'logreros', '\n', 'dicta', 'lecciones', 'de', 'caballería', '.', '\n', 'y', 'el', 'alma', 'desalmada', 'de', 'su', 'raza', ',', '\n', 'que', 'bajo', 'el', 'golpe', 'de', 'su', 'férrea', 'maza', '\n', 'aún', 'durme', ',', 'puede', 'que', 'despierte', 'un', 'día', '.', '\n', 'quiere', 'enseñar', 'el', 'ceño', 'de', 'la', 'duda', ',', '\n', 'antes', 'de',

In [4]:
with open('word_idx.pkl', 'rb') as file:
    word_idx = pickle.load(file)
    
with open('idx_word.pkl', 'rb') as file:
    idx_word = pickle.load(file)

vocabulary_size = len(word_idx) + 1
print(vocabulary_size)

6299


In [5]:
tokenized = []
for poem in poems:
    tokenized.append([word_idx[word] for word in poem])

print(poems[0])
print(tokenized[0])
print("---")
print(poems[0][7], "->", word_idx[poems[0][7]])

['a', 'don', 'miguel', 'de', 'unamuno', '\n', 'por', 'su', 'libro', 'vida', 'de', 'don', 'quijote', 'y', 'sancho', '.', '\n', 'este', 'donquijotesco', '\n', 'don', 'miguel', 'de', 'unamuno', ',', 'fuerte', 'vasco', ',', '\n', 'lleva', 'el', 'arnés', 'grotesco', '\n', 'y', 'el', 'irrisorio', 'casco', '\n', 'del', 'buen', 'manchego', '.', 'don', 'miguel', 'camina', ',', '\n', 'jinete', 'de', 'quimérica', 'montura', ',', '\n', 'metiendo', 'espuela', 'de', 'oro', 'a', 'su', 'locura', ',', '\n', 'sin', 'miedo', 'de', 'la', 'lengua', 'que', 'malsina', '.', '\n', 'a', 'un', 'pueblo', 'de', 'arrieros', ',', '\n', 'lechuzos', 'y', 'tahúres', 'y', 'logreros', '\n', 'dicta', 'lecciones', 'de', 'caballería', '.', '\n', 'y', 'el', 'alma', 'desalmada', 'de', 'su', 'raza', ',', '\n', 'que', 'bajo', 'el', 'golpe', 'de', 'su', 'férrea', 'maza', '\n', 'aún', 'durme', ',', 'puede', 'que', 'despierte', 'un', 'día', '.', '\n', 'quiere', 'enseñar', 'el', 'ceño', 'de', 'la', 'duda', ',', '\n', 'antes', 'de',

In [6]:
lengths = [len(sequence) for sequence in tokenized]
print(max(lengths))
print(min(lengths))

4758
13


In [7]:
max_seq_length = 600 #4758

padded = []
for sequence in tokenized:
    trimmed = sequence[-max_seq_length:]
    padding = [0] * (max_seq_length - len(trimmed))
    padded.append(padding + trimmed)
    
padded = np.array(padded)
print(padded.shape, padded[:,:-1].shape, padded[:,1:].shape)

(445, 600) (445, 599) (445, 599)


In [0]:
# create data loader
batch_size = 4

dataset = TensorDataset(torch.from_numpy(padded[:,:-1]), torch.from_numpy(padded[:,1:]))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [9]:
x, y = next(iter(dataloader))

print(x.shape, y.shape)

print(x[0, -10:])
print(y[0, -10:])

torch.Size([4, 599]) torch.Size([4, 599])
tensor([  86, 5342,   14,   15, 5343, 5344,    4,  275,   14,    1])
tensor([5342,   14,   15, 5343, 5344,    4,  275,   14,    1,   24])


In [0]:
model = transformer.Transformer(vocabulary_size, 512, max_seq_length-1, blocks=12, heads=8)
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)
# model

In [56]:
# training
epochs = 10

optimizer = torch.optim.Adam(model.parameters(), lr=0.00001, betas=(0.9, 0.98), eps=1e-9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

model.cuda()
model.train()
for e in range(1, epochs+1):
    scheduler.step()
    total_loss = 0
    total_accuracy = 0
    batch = 0
    for x, y in dataloader:
        x, y = x.cuda(), y.cuda()
        batch += 1
        optimizer.zero_grad()
    
        mask = transformer.gen_target_mask(x.cpu(), 0).cuda()
    
        preds = model.encoder(x, mask)
        preds = model.out(preds)
        
        loss = F.cross_entropy(preds.view(-1, preds.size(-1)), y.view(-1)) #, ignore_index=0)
        total_loss += loss.item()
    
        loss.backward()
        optimizer.step()
        
        equals = torch.argmax(F.softmax(preds, dim=-1), dim=-1).view(-1) == y.view(-1)
        total_accuracy += torch.mean(equals.type(torch.FloatTensor))
    
        if batch % 14 == 0:
            print(f"EPOCH {e} ({batch}/{len(dataloader)}) - loss {total_loss/batch:.4f} - acc {total_accuracy/batch:.4f}") 

    print(f"EPOCH {e} - loss {total_loss/len(dataloader):.4f} - acc {total_accuracy/len(dataloader):.4f} ---------------------- ")

EPOCH 1 (14/112) - loss 0.0215 - acc 0.9953
EPOCH 1 (28/112) - loss 0.0207 - acc 0.9958
EPOCH 1 (42/112) - loss 0.0205 - acc 0.9958
EPOCH 1 (56/112) - loss 0.0205 - acc 0.9959
EPOCH 1 (70/112) - loss 0.0204 - acc 0.9959
EPOCH 1 (84/112) - loss 0.0202 - acc 0.9959
EPOCH 1 (98/112) - loss 0.0201 - acc 0.9960
EPOCH 1 (112/112) - loss 0.0194 - acc 0.9961
EPOCH 1 - loss 0.0194 - acc 0.9961 ---------------------- 
EPOCH 2 (14/112) - loss 0.0140 - acc 0.9970
EPOCH 2 (28/112) - loss 0.0151 - acc 0.9969
EPOCH 2 (42/112) - loss 0.0156 - acc 0.9968
EPOCH 2 (56/112) - loss 0.0154 - acc 0.9969
EPOCH 2 (70/112) - loss 0.0156 - acc 0.9968
EPOCH 2 (84/112) - loss 0.0153 - acc 0.9968
EPOCH 2 (98/112) - loss 0.0154 - acc 0.9968
EPOCH 2 (112/112) - loss 0.0157 - acc 0.9968
EPOCH 2 - loss 0.0157 - acc 0.9968 ---------------------- 
EPOCH 3 (14/112) - loss 0.0145 - acc 0.9971
EPOCH 3 (28/112) - loss 0.0135 - acc 0.9973
EPOCH 3 (42/112) - loss 0.0138 - acc 0.9974
EPOCH 3 (56/112) - loss 0.0140 - acc 0.9973


In [0]:
torch.save(model.state_dict(), 'drive/My Drive/Colab Notebooks/models/gpt_machado.pt')

In [0]:
checkpoint = torch.load('drive/My Drive/Colab Notebooks/models/gpt_machado.pt')
model.load_state_dict(checkpoint)

In [58]:
# inference
seed = 'ojos azules como el agua \n almas solas en tormento \n la calle desierta \n ¡ un amor ardiente ! \n'
seed = [word_idx[word] for word in seed.split(' ')]

def sample(softmax_logits, temperature=1.0):
    tempered = F.softmax(softmax_logits / temperature, dim=-1)
    distribution = torch.multinomial(tempered, 1)
    return distribution[0]

model.cuda()
model.eval()
with torch.no_grad():
    for i in range(300):
        x = torch.from_numpy(np.array(seed)).unsqueeze(0).cuda()
        mask = transformer.gen_target_mask(x.cpu(), 0).cuda()
        encoded = model.encoder(x, mask)
        out = model.out(encoded)
        
        #out_softmax = F.softmax(out, dim=-1)
        # idx = torch.argmax(out_softmax, dim=-1)[:, -1].item()
        
        idx = sample(out[:, -1], temperature=0.7).item()
        
        seed.append(idx)
        if idx_word == 0:
            break
        word = idx_word[idx]
        if word == '<end>':
            break

print(' '.join([idx_word[idx] for idx in seed]))

ojos azules como el agua 
 almas solas en tormento 
 la calle desierta 
 ¡ un amor ardiente ! 
 cerca del duero arriba . 
 la voz que la mar se le ilumina , 
 la torre castellana . 
 la hora , 
 sólo el río , 
 junto al agua que hizo un « algunas nubes plomizas 
 como un jardín sombrío , 
 como un « porque aguardo y el puro , 
 hacia la fuente . 
 hacia la tierra sombrío 
 ya sólo el pan con otras doncellitas 
 en los días azules y , 
 cuando crecen las blancas margaritas . 
 vi 
 ¡ oh dulce y la mar ! 
 cabeza de extremadura , 
 con su poma , 
 arruinado , 
 estos limonares verdes 
 con sus hojas , de sus casas denegridas ! 
 ¡ oh , 
 ii 
 y es del agua limpia : 
 la oveja pace , 
 por ti y siempre espera , 
 en sus galgos , 
 de españa , que tiene 
 en torno de fe sin verdor ; 
 con tu tronco ceniciento 
 sin esbeltez ni canciones 
 cuando graznan las cornejas ! 
 y siempre o aparece 
 como tus largos ríos , 
 hacia la mar ! 
 soria , sobrado lacónico . 
 ¿ acaso os asombra mi corazó